# **Experiment 3**

In [1]:
!pip install keras
!pip install emoji --upgrade
!pip install keras-self-attention
!pip install wordninja

     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=95372066a9e41dcfc7c64c8841d9b00d80f958d74e035f8fa17b096cd568a087
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
  Created wheel for keras-self-attention: filename=keras_self_attention-0.42.0-cp36-none-any.whl size=17296 sha256=58445a39c0d6ac12dac8bc8fb656a8d645b122edf703dac3eb045204d401a02b
  Stored in directory: /root/.cache/pip/wheels/7b/05/a0/99c0cf60d383f0494e10eca2b238ea98faca9a1fe03cac2894
Successfully built keras-self-attention
     |████████████████████████████████| 542kB 4.8MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp36-none-any.whl size=541552 sha256=32c832e3b97742dafbbc98779cdfa1b815bdddccc8052a7f65560f50f85c175c
  Stored in directory: /root/.cache/pip/wheels/22/46/06/9b6d10ed02c85e93c3bb33ac50e2d368b2586248f192a2e22a
Successfully built

# **Headers**

In [2]:
##################Headers#########
import pandas as pd
import keras
import emoji
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


# **Loading the Data**

In [0]:
classes_list = ["NOT","HOF"]
df_train["class"]=df_train['task_1'].apply(classes_list.index)
df_test["class"]=df_test['task_1'].apply(classes_list.index)

# ***Preprocessing Functions***

Expanding Contractions


In [0]:
import re
#########Contraction List#####
cList = {"ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have",
"couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not",
"hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have",
"he's": "he is","how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have",
"I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not","it'd": "it had","it'd've": "it would have",
"it'll": "it will","it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
"might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not",
"mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have","she'd": "she would",
"she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is","should've": "should have",
"shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would",
"that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are",
"they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had","we'd've": "we would have","we'll": "we will",
"we'll've": "we will have","we're": "we are","we've": "we have","weren't": "were not","what'll": "what will","what'll've": "what will have",
"what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have","where'd": "where did",
"where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have",
"why's": "why is","why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have",
"wouldn't": "would not","wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
"y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have",
"you'll": "you you will","you'll've": "you you will have","you're": "you are","you've": "you have"}
#########Function for contraction#####
c_re = re.compile('(%s)' % '|'.join(cList.keys())) #regular expression object, with format of contractions like i'm, i've
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)] #find the match in the list
    return c_re.sub(replace, text)  #replacing the short hands with expanded version

Lemmatization according to sentence structure 

In [7]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
from nltk.corpus import stopwords
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
lemma_function = WordNetLemmatizer()
def lemmatize(row):
  tokens = word_tokenize(row)
  new =  ""
  for token, tag in pos_tag(tokens):
    if token not in stopwords.words('english'):
      lemma = lemma_function.lemmatize(token, tag_map[tag[0]])
      new=new+lemma+" "
  return new

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# **Preprocessing**

In [0]:
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
df_train["text"]=df_train["text"].str.replace("@","username") #removing @
df_train["text"]=df_train["text"].str.replace("'s","")
df_train["text"]=df_train["text"].str.replace("?"," question ")
df_train["text"]=df_train["text"].str.replace("!"," exclamation ")
df_train["text"]=df_train["text"].str.replace(r'http\S+', '')# removing links
df_train["text"]=df_train["text"].apply(expandContractions) #expanding contractions
df_train["text"]=df_train["text"].apply(emoji.demojize)#changing emojis to text
df_train["text"]=df_train["text"].str.replace("#","").str.replace("_"," ").str.replace(":"," ").str.lower()
df_train["text"]=df_train["text"].apply(lemmatize) #lemmatization
df_train["text"]=df_train["text"].str.replace(r'[^\w\s]'," ") #removing punctuation
df_test["text"]=df_test["text"].str.replace("@","username")
df_test["text"]=df_test["text"].str.replace("'s","")
df_test["text"]=df_test["text"].str.replace("?"," question ")
df_test["text"]=df_test["text"].str.replace("!"," exclamation ")
df_test["text"]=df_test["text"].apply(split_hashtag)
df_test["text"]=df_test["text"].str.replace(r'http\S+', '')
df_test["text"]=df_test["text"].apply(expandContractions)
df_test["text"]=df_test["text"].apply(emoji.demojize)
df_test["text"]=df_test["text"].str.replace("#","").str.replace("_"," ").str.replace(":","").str.lower()
df_test["text"]=df_test["text"].apply(lemmatize)
df_test["text"]=df_test["text"].str.replace(r'[^\w\s]',"")
tokenizer = Tokenizer(num_words=5000) # tokenizing
tokenizer.fit_on_texts(df_train["text"].values.tolist())
#Creating the training and testing sets consisting of tokens
X_train = tokenizer.texts_to_sequences(df_train["text"].values.tolist())
X_test = tokenizer.texts_to_sequences(df_test["text"].values.tolist())
vocab_size = len(tokenizer.word_index) + 1 
word_index=tokenizer.word_index

Preparing the training and testing sets for Classification

In [0]:
maxlen=max([len(a) for a in X_train])

In [0]:
from keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
y_train=df_train["class"].values.tolist()

# **Function for Embedding**

In [0]:
import numpy as np
def make_glovevec(glovepath, max_features, embed_size, word_index, veclen=300):
    embeddings_index = {}
    f = open(glovepath)
    for line in f:
        values = line.split()
        dimension=len(values)-1
        word = ' '.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs.reshape(-1)
    f.close()

    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
max_features=len(word_index)
embed_size=300
embedding_vector = make_glovevec("/content/drive/My Drive/english_dataset/glove.840B.300d.txt", max_features+1, embed_size, word_index)

# The Classifiers 

In [0]:
 ############### Keras
from keras.models import Sequential
from keras_self_attention import SeqSelfAttention as Attention
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [19]:
#################BiLSTM Attention+ Relu without any dropouts ######################
d2=0.0
d1=0.0
model_glove = Sequential()
model_glove.add(Embedding(len(word_index)+1, 300, input_length=129, weights=[embedding_vector], trainable=False))
model_glove.add(Bidirectional(LSTM(300, return_sequences=True, dropout=d1,recurrent_dropout=d2)))
model_glove.add(Attention(attention_activation='sigmoid'))
model_glove.add(Flatten())
model_glove.add(Dense(10, activation='relu'))
model_glove.add(Dropout(0.0)) 
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(X_train, y_train, validation_split=0.2, epochs = 10)
y_pred = model_glove.predict_classes(X_test)
y_test=df_test["class"].values.tolist()
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))   
confusion_matrix(y_test, y_pred)   

Train on 4681 samples, validate on 1171 samples
Epoch 1/10
4681/4681 [==============================] - 243s 52ms/step - loss: 0.5850 - accuracy: 0.7195 - val_loss: 0.8603 - val_accuracy: 0.3928
Epoch 2/10
4681/4681 [==============================] - 242s 52ms/step - loss: 0.5371 - accuracy: 0.7411 - val_loss: 1.0411 - val_accuracy: 0.3518
Epoch 3/10
4681/4681 [==============================] - 244s 52ms/step - loss: 0.5154 - accuracy: 0.7522 - val_loss: 0.9542 - val_accuracy: 0.4210
Epoch 4/10
4681/4681 [==============================] - 241s 52ms/step - loss: 0.4965 - accuracy: 0.7599 - val_loss: 1.0315 - val_accuracy: 0.4116
Epoch 5/10
4681/4681 [==============================] - 243s 52ms/step - loss: 0.4739 - accuracy: 0.7802 - val_loss: 1.2553 - val_accuracy: 0.3911
Epoch 6/10
4681/4681 [==============================] - 247s 53ms/step - loss: 0.4561 - accuracy: 0.7868 - val_loss: 1.0087 - val_accuracy: 0.3911
Epoch 7/10
4681/4681 [==============================] - 239s 51ms/step

# **Extras**

In [20]:
#################BiLSTM Attention without any dropouts ######################
d2=0.0
d1=0.0
model_glove = Sequential()
model_glove.add(Embedding(len(word_index)+1, 300, input_length=129, weights=[embedding_vector], trainable=False))
model_glove.add(Bidirectional(LSTM(300, return_sequences=True, dropout=d1,recurrent_dropout=d2)))
model_glove.add(Attention(attention_activation='sigmoid'))
model_glove.add(Flatten())
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(X_train, y_train, validation_split=0.2, epochs = 10)
y_pred = model_glove.predict_classes(X_test)
y_test=df_test["class"].values.tolist()
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))      

Train on 4681 samples, validate on 1171 samples
Epoch 1/10
4681/4681 [==============================] - 254s 54ms/step - loss: 0.5797 - accuracy: 0.7090 - val_loss: 0.9242 - val_accuracy: 0.4142
Epoch 2/10
4681/4681 [==============================] - 243s 52ms/step - loss: 0.5352 - accuracy: 0.7419 - val_loss: 0.9968 - val_accuracy: 0.3749
Epoch 3/10
4681/4681 [==============================] - 248s 53ms/step - loss: 0.5158 - accuracy: 0.7601 - val_loss: 0.9396 - val_accuracy: 0.4031
Epoch 4/10
4681/4681 [==============================] - 243s 52ms/step - loss: 0.5003 - accuracy: 0.7631 - val_loss: 1.2389 - val_accuracy: 0.3723
Epoch 5/10
4681/4681 [==============================] - 239s 51ms/step - loss: 0.4740 - accuracy: 0.7746 - val_loss: 1.1495 - val_accuracy: 0.3749
Epoch 6/10
4681/4681 [==============================] - 223s 48ms/step - loss: 0.4344 - accuracy: 0.7934 - val_loss: 1.2238 - val_accuracy: 0.4082
Epoch 7/10
4681/4681 [==============================] - 222s 47ms/step

In [21]:
#################BiLSTM Attention Dropouts-0.2,0.1,0.05######################
d2=0.2
d1=0.1
model_glove = Sequential()
model_glove.add(Embedding(len(word_index)+1, 300, input_length=129, weights=[embedding_vector], trainable=False))
model_glove.add(Bidirectional(LSTM(300, return_sequences=True, dropout=d1,recurrent_dropout=d2)))
model_glove.add(Attention(attention_activation='sigmoid'))
model_glove.add(Flatten())
model_glove.add(Dense(10, activation='relu'))
model_glove.add(Dropout(0.05)) 
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(X_train, y_train, validation_split=0.2, epochs = 6)
y_pred = model_glove.predict_classes(X_test)
y_test=df_test["class"].values.tolist()
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))      

Train on 4681 samples, validate on 1171 samples
Epoch 1/6
4681/4681 [==============================] - 229s 49ms/step - loss: 0.5881 - accuracy: 0.7125 - val_loss: 0.8163 - val_accuracy: 0.4910
Epoch 2/6
4681/4681 [==============================] - 228s 49ms/step - loss: 0.5437 - accuracy: 0.7349 - val_loss: 0.9642 - val_accuracy: 0.3860
Epoch 3/6
4681/4681 [==============================] - 232s 49ms/step - loss: 0.5263 - accuracy: 0.7424 - val_loss: 0.9707 - val_accuracy: 0.4039
Epoch 4/6
4681/4681 [==============================] - 232s 50ms/step - loss: 0.5143 - accuracy: 0.7535 - val_loss: 1.1224 - val_accuracy: 0.3775
Epoch 5/6
4681/4681 [==============================] - 229s 49ms/step - loss: 0.5039 - accuracy: 0.7571 - val_loss: 1.0750 - val_accuracy: 0.3424
Epoch 6/6
4681/4681 [==============================] - 233s 50ms/step - loss: 0.4827 - accuracy: 0.7695 - val_loss: 0.8940 - val_accuracy: 0.3834
Accuracy score = 0.8230702515177797
              precision    recall  f1-sc

In [22]:
#################BiLSTM Attention Dropouts-0.2,0.1,0.05######################
d2=0.0
d1=0.0
model_glove = Sequential()
model_glove.add(Embedding(len(word_index)+1, 300, input_length=129, weights=[embedding_vector], trainable=False))
model_glove.add(Bidirectional(LSTM(300, return_sequences=True, dropout=d1,recurrent_dropout=d2)))
model_glove.add(Attention(attention_activation='sigmoid'))
model_glove.add(Flatten())
#model_glove.add(Dense(10, activation='relu'))
#model_glove.add(Dropout(0.05)) 
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(X_train, y_train, validation_split=0.2, epochs = 6)
y_pred = model_glove.predict_classes(X_test)
y_test=df_test["class"].values.tolist()
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))      

Train on 4681 samples, validate on 1171 samples
Epoch 1/6
4681/4681 [==============================] - 227s 48ms/step - loss: 0.5773 - accuracy: 0.7223 - val_loss: 0.9363 - val_accuracy: 0.3817
Epoch 2/6
4681/4681 [==============================] - 234s 50ms/step - loss: 0.5351 - accuracy: 0.7454 - val_loss: 0.9259 - val_accuracy: 0.3945
Epoch 3/6
4681/4681 [==============================] - 228s 49ms/step - loss: 0.5169 - accuracy: 0.7550 - val_loss: 1.1660 - val_accuracy: 0.3621
Epoch 4/6
4681/4681 [==============================] - 227s 48ms/step - loss: 0.5041 - accuracy: 0.7631 - val_loss: 1.0445 - val_accuracy: 0.4082
Epoch 5/6
4681/4681 [==============================] - 228s 49ms/step - loss: 0.4717 - accuracy: 0.7791 - val_loss: 1.0439 - val_accuracy: 0.3945
Epoch 6/6
4681/4681 [==============================] - 226s 48ms/step - loss: 0.4452 - accuracy: 0.7973 - val_loss: 1.0614 - val_accuracy: 0.4022
Accuracy score = 0.8100607111882047
              precision    recall  f1-sc

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[808,  57],
       [162, 126]])

In [24]:
d2=0.2
d1=0.25
model_glove = Sequential()
model_glove.add(Embedding(len(word_index)+1, 300, input_length=129, weights=[embedding_vector], trainable=False))
model_glove.add(Bidirectional(LSTM(300, return_sequences=True, dropout=d1,recurrent_dropout=d2)))
model_glove.add(Attention(attention_activation='sigmoid'))
model_glove.add(Flatten())
model_glove.add(Dense(10, activation='relu'))
model_glove.add(Dropout(0.1)) 
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(X_train, y_train, validation_split=0.2, epochs = 5)
y_pred = model_glove.predict_classes(X_test)
y_test=df_test["class"].values.tolist()
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))      

Train on 4681 samples, validate on 1171 samples
Epoch 1/5
4681/4681 [==============================] - 231s 49ms/step - loss: 0.5899 - accuracy: 0.7020 - val_loss: 0.9900 - val_accuracy: 0.2921
Epoch 2/5
4681/4681 [==============================] - 231s 49ms/step - loss: 0.5516 - accuracy: 0.7272 - val_loss: 0.9772 - val_accuracy: 0.3348
Epoch 3/5
4681/4681 [==============================] - 230s 49ms/step - loss: 0.5423 - accuracy: 0.7278 - val_loss: 1.0722 - val_accuracy: 0.3091
Epoch 4/5
4681/4681 [==============================] - 232s 50ms/step - loss: 0.5313 - accuracy: 0.7432 - val_loss: 0.9881 - val_accuracy: 0.3365
Epoch 5/5
4681/4681 [==============================] - 231s 49ms/step - loss: 0.5206 - accuracy: 0.7398 - val_loss: 0.9451 - val_accuracy: 0.3766
Accuracy score = 0.8074588031222897
              precision    recall  f1-score   support

           0       0.81      0.97      0.88       865
           1       0.77      0.33      0.46       288

    accuracy          

In [25]:
##########LSTM+CNN-Modified features#########
model_glove = Sequential()
model_glove.add(Embedding(len(word_index)+1, 300, input_length=129, weights=[embedding_vector], trainable=False))
model_glove.add(Conv1D(32, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(300, return_sequences=True))#, dropout=0.05,recurrent_dropout=0.05))
model_glove.add(Flatten())
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
y_train=df_train["class"].values.tolist()
model_glove.fit(X_train, y_train, validation_split=0.2, epochs = 5)
y_pred = model_glove.predict_classes(X_test)
y_test=df_test["class"].values.tolist()
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))      
confusion_matrix(y_test, y_pred)

Train on 4681 samples, validate on 1171 samples
Epoch 1/5
4681/4681 [==============================] - 24s 5ms/step - loss: 0.5761 - accuracy: 0.7127 - val_loss: 0.8851 - val_accuracy: 0.3740
Epoch 2/5
4681/4681 [==============================] - 24s 5ms/step - loss: 0.5030 - accuracy: 0.7620 - val_loss: 1.1924 - val_accuracy: 0.3595
Epoch 3/5
4681/4681 [==============================] - 24s 5ms/step - loss: 0.4316 - accuracy: 0.8097 - val_loss: 1.0580 - val_accuracy: 0.4048
Epoch 4/5
4681/4681 [==============================] - 23s 5ms/step - loss: 0.3197 - accuracy: 0.8705 - val_loss: 0.9961 - val_accuracy: 0.5260
Epoch 5/5
4681/4681 [==============================] - 24s 5ms/step - loss: 0.2326 - accuracy: 0.9124 - val_loss: 1.5197 - val_accuracy: 0.4663
Accuracy score = 0.779705117085863
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       865
           1       0.55      0.61      0.58       288

    accuracy                     

array([[724, 141],
       [113, 175]])

In [26]:
##########LSTM+CNN-Modified features#########
model_glove = Sequential()
model_glove.add(Embedding(len(word_index)+1, 300, input_length=129, weights=[embedding_vector], trainable=False))
model_glove.add(Conv1D(32, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(300, return_sequences=True, dropout=0.2,recurrent_dropout=0.1))
model_glove.add(Flatten())
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
y_train=df_train["class"].values.tolist()
model_glove.fit(X_train, y_train, validation_split=0.2, epochs = 5)
y_pred = model_glove.predict_classes(X_test)
y_test=df_test["class"].values.tolist()
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))      
confusion_matrix(y_test, y_pred)

Train on 4681 samples, validate on 1171 samples
Epoch 1/5
4681/4681 [==============================] - 25s 5ms/step - loss: 0.5828 - accuracy: 0.7112 - val_loss: 0.8565 - val_accuracy: 0.3860
Epoch 2/5
4681/4681 [==============================] - 24s 5ms/step - loss: 0.5265 - accuracy: 0.7477 - val_loss: 0.9039 - val_accuracy: 0.4253
Epoch 3/5
4681/4681 [==============================] - 24s 5ms/step - loss: 0.4708 - accuracy: 0.7787 - val_loss: 0.8216 - val_accuracy: 0.4586
Epoch 4/5
4681/4681 [==============================] - 24s 5ms/step - loss: 0.4101 - accuracy: 0.8088 - val_loss: 1.0679 - val_accuracy: 0.4330
Epoch 5/5
4681/4681 [==============================] - 24s 5ms/step - loss: 0.3473 - accuracy: 0.8541 - val_loss: 1.0971 - val_accuracy: 0.4492
Accuracy score = 0.7623590633130962
              precision    recall  f1-score   support

           0       0.85      0.84      0.84       865
           1       0.52      0.54      0.53       288

    accuracy                    

array([[723, 142],
       [132, 156]])

In [29]:
#################BiLSTM Attention+ Relu without any dropouts ######################
d2=0.0
d1=0.0
model_glove = Sequential()
model_glove.add(Embedding(len(word_index)+1, 300, input_length=129, weights=[embedding_vector], trainable=False))
model_glove.add(Bidirectional(LSTM(300, return_sequences=True, dropout=d1,recurrent_dropout=d2)))
model_glove.add(Attention(attention_activation='sigmoid'))
model_glove.add(Flatten())
model_glove.add(Dense(10, activation='relu'))
model_glove.add(Dropout(0.0)) 
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(X_train, y_train, validation_split=0.2, epochs = 7)
y_pred = model_glove.predict_classes(X_test)
y_test=df_test["class"].values.tolist()
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))   
confusion_matrix(y_test, y_pred)   

Train on 4681 samples, validate on 1171 samples
Epoch 1/7
4681/4681 [==============================] - 227s 48ms/step - loss: 0.5802 - accuracy: 0.7131 - val_loss: 1.0040 - val_accuracy: 0.3279
Epoch 2/7
4681/4681 [==============================] - 224s 48ms/step - loss: 0.5375 - accuracy: 0.7347 - val_loss: 1.0232 - val_accuracy: 0.3587
Epoch 3/7
4681/4681 [==============================] - 228s 49ms/step - loss: 0.5145 - accuracy: 0.7477 - val_loss: 0.9048 - val_accuracy: 0.3416
Epoch 4/7
4681/4681 [==============================] - 223s 48ms/step - loss: 0.5021 - accuracy: 0.7539 - val_loss: 0.9867 - val_accuracy: 0.3757
Epoch 5/7
4681/4681 [==============================] - 225s 48ms/step - loss: 0.4772 - accuracy: 0.7742 - val_loss: 1.2079 - val_accuracy: 0.3621
Epoch 6/7
4681/4681 [==============================] - 227s 48ms/step - loss: 0.4591 - accuracy: 0.7795 - val_loss: 1.0818 - val_accuracy: 0.4278
Epoch 7/7
4681/4681 [==============================] - 225s 48ms/step - loss

array([[792,  73],
       [142, 146]])

In [30]:
#################BiLSTM Attention Dropouts-0.2,0.05,0.05######################
d2=0.2
d1=0.05
model_glove = Sequential()
model_glove.add(Embedding(len(word_index)+1, 300, input_length=129, weights=[embedding_vector], trainable=False))
model_glove.add(Bidirectional(LSTM(300, return_sequences=True, dropout=d1,recurrent_dropout=d2)))
model_glove.add(Attention(attention_activation='sigmoid'))
model_glove.add(Flatten())
model_glove.add(Dense(10, activation='relu'))
model_glove.add(Dropout(0.05)) 
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(X_train, y_train, validation_split=0.2, epochs = 10)
y_pred = model_glove.predict_classes(X_test)
y_test=df_test["class"].values.tolist()
print("Accuracy score =", accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))      

Train on 4681 samples, validate on 1171 samples
Epoch 1/10
4681/4681 [==============================] - 230s 49ms/step - loss: 0.5960 - accuracy: 0.6981 - val_loss: 0.9824 - val_accuracy: 0.3493
Epoch 2/10
4681/4681 [==============================] - 230s 49ms/step - loss: 0.5566 - accuracy: 0.7052 - val_loss: 1.0724 - val_accuracy: 0.2750
Epoch 3/10
4681/4681 [==============================] - 229s 49ms/step - loss: 0.5398 - accuracy: 0.6984 - val_loss: 0.9643 - val_accuracy: 0.2750
Epoch 4/10
4681/4681 [==============================] - 233s 50ms/step - loss: 0.5330 - accuracy: 0.7287 - val_loss: 0.9721 - val_accuracy: 0.4620
Epoch 5/10
4681/4681 [==============================] - 231s 49ms/step - loss: 0.5103 - accuracy: 0.7537 - val_loss: 0.8975 - val_accuracy: 0.4330
Epoch 6/10
4681/4681 [==============================] - 230s 49ms/step - loss: 0.4850 - accuracy: 0.7695 - val_loss: 1.0904 - val_accuracy: 0.4535
Epoch 7/10
4681/4681 [==============================] - 232s 50ms/step